# Analyzing Students' Mental Health

InIn this project, I analyze a dataset of students in a Japanese university to understand how the length of stay impacts tirhe average mental healt Mental health is evaluated based on the following metrics: depression, social connectedness (extent to which individuals feel a sense of belonging in a group) and acculturative stress (stress associated with adapting to a new culture).



The idea and dataset for this project are from [this DataCamp project](https://app.datacamp.com/learn/projects/analyzing_students_mental_health/guided/SQL).

## The dataset

The data can be found in the `students.csv` file. Below is a description of the more relevant columns to our analysis.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

## Load the `sql` magic object and connect the `duckdb` database engine

In [1]:
# Load `sql` object
%load_ext sql

In [2]:
# Connect `duckdb` engine
%sql duckdb://

Connecting to 'duckdb://'

## Exploratory data analysis

First, let's have an idea of our entire data.

In [3]:
%%sql 

# Inspect data
SELECT *
FROM students.csv
LIMIT 5;

Running query in 'duckdb://'

inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,None,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,None,Yes,Yes,Yes,No,No,No,No,No,No,No,No
Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,None,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,None,Yes,Yes,Yes,No,No,No,No,No,No,No,No
Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,None,No,No,No,No,No,No,No,No,No,No,No
Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,None,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,None,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


Let's see the unique values in the `inter_dom` field, which says whether a student is a domestic or international students.

In [11]:
%%sql

SELECT DISTINCT inter_dom
FROM students.csv;

Running query in 'duckdb://'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

inter_dom
Dom
Inter
None


There's actually no 'None' value, contrary to what the query result above suggests. See the next two code cells.

In [16]:
%%sql

SELECT COUNT(inter_dom)
FROM students.csv
WHERE inter_dom = 'None';

Running query in 'duckdb://'

count(inter_dom)
0


In [18]:
%%sql

SELECT COUNT(DISTINCT inter_dom)
FROM students.csv;

Running query in 'duckdb://'

count(DISTINCT inter_dom)
2


`Inter` suggests that the student is an international student. `Dom` implies they are a domestic student.

## Evaluate the impact of students' length of stay and status (international or domestic) on their mental health

In [38]:
%%sql

SELECT
	stay, inter_dom,
	COUNT(inter_dom) count_stud, 
	ROUND(AVG(todep), 2) average_phq, 
	ROUND(AVG(tosc), 2) average_scs, 
	ROUND(AVG(toas), 2) average_as
FROM students.csv
WHERE stay IS NOT NULL
GROUP BY stay, inter_dom
ORDER BY stay DESC;

Running query in 'duckdb://'

stay,inter_dom,count_stud,average_phq,average_scs,average_as
10,Inter,1,13.0,32.0,50.0
8,Inter,1,10.0,44.0,65.0
7,Inter,1,4.0,48.0,45.0
6,Inter,3,6.0,38.0,58.67
5,Dom,2,11.5,34.0,88.0
5,Inter,1,0.0,34.0,91.0
4,Dom,9,7.0,36.67,64.78
4,Inter,14,8.57,33.93,87.71
3,Dom,23,8.43,39.09,58.04
3,Inter,46,9.09,37.13,78.0


Since the count of students where length of stay is over 4 years is very small (1-3), the mental diagnostic scores may not be representative of the whole sample.

Let's zoom in to records where length of stay is at least 4.

In [40]:
%%sql

SELECT
	stay, inter_dom,
	COUNT(inter_dom) count_stud, 
	ROUND(AVG(todep), 2) average_phq, 
	ROUND(AVG(tosc), 2) average_scs, 
	ROUND(AVG(toas), 2) average_as
FROM students.csv
WHERE stay IS NOT NULL AND stay < 5
GROUP BY stay, inter_dom
ORDER BY stay DESC;

Running query in 'duckdb://'

stay,inter_dom,count_stud,average_phq,average_scs,average_as
4,Inter,14,8.57,33.93,87.71
4,Dom,9,7.0,36.67,64.78
3,Inter,46,9.09,37.13,78.0
3,Dom,23,8.43,39.09,58.04
2,Dom,13,9.46,37.08,66.46
2,Inter,39,8.28,37.08,77.67
1,Inter,95,7.48,38.11,72.8
1,Dom,20,8.7,37.15,62.6


The effect of year of stay and/or student status (international or domestic) on students' mental health seems inconclusive.